# Sports Analytics Project

## Analysis on IPL - Dataset [2008 - 2024]

### Coverings :
### This Notebook Will Provide

- Indepth Analysis of Players Performances
- Time Series Analysis on Boundries , Runs , Catches Across every season
- Moving / Rolling Average of Runs Scores by every batsmen & eceonmy rate of every bowler
- Team Metrics / Comaprision
- Playes Comparision [Inter-team / Intra-team]

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
match = pd.read_csv('/kaggle/input/ipl-complete-dataset-20082020/matches.csv')
delivery = pd.read_csv('/kaggle/input/ipl-complete-dataset-20082020/deliveries.csv')

In [ ]:
match.info()

In [ ]:
delivery.info()

In [ ]:
match.sample(7)

In [ ]:
delivery.sample(7)

## EDA on Matches & delieveries Data

In [ ]:
match[['id']].describe()

In [ ]:
match['season'].value_counts().sort_index()

# Data Cleaning Steps [For Documentation Purposes]

- Match Seasons [2009/10 -> 2010 , 2020/21 -> 2020]
- Removing Duplicate Team Names & grouping into 1

In [ ]:
change_season1 = match[match['season'] == '2007/08'].index
change_season2 = match[match['season'] == '2009/10'].index
change_season3 = match[match['season'] == '2020/21'].index

In [ ]:
match.loc[change_season1,"season"] = "2008"
match.loc[change_season2,"season"] = "2010"
match.loc[change_season3,"season"] = "2020"

In [ ]:
match['season'].value_counts().sort_index()

In [ ]:
match['city'].value_counts()

In [ ]:
match['match_type'].value_counts()

In [ ]:
# Eliminating Redundant Team Names
(match['team1'].value_counts() + match['team2'].value_counts()).sort_index().index

In [ ]:
d = {}
d['Chennai Super Kings'] = "csk"
d['Delhi Capitals'] = "dc"
d['Delhi Daredevils'] = "dc"
d['Gujarat Lions'] = "gt"
d['Gujarat Titans'] = "gt"
d['Kings XI Punjab'] = "pbks"
d['Kolkata Knight Riders'] = "kkr"
d['Lucknow Super Giants'] = "lsg"
d['Mumbai Indians'] = "mi"
d['Pune Warriors'] = "rpsg"
d['Punjab Kings'] = "pbks"
d['Rajasthan Royals'] = "rr"
d['Rising Pune Supergiant'] = "rpsg"
d['Royal Challengers Bangalore'] = "rcb"
d['Royal Challengers Bengaluru'] = "rcb"
d['Sunrisers Hyderabad'] = "srh"
d['Deccan Chargers'] = "srh"
d['Kochi Tuskers Kerala'] = "ktk"
d['Rising Pune Supergiants'] = "rpsg"

In [ ]:
match['team1'] = match['team1'].replace(d)

In [ ]:
match['team2'] = match['team2'].replace(d)

In [ ]:
match['winner'] = match['winner'].replace(d)

In [ ]:
match['toss_winner'] = match['winner'].replace(d)

In [ ]:
match

In [ ]:
match['player_of_match'].value_counts()

In [ ]:
match['method'].value_counts()

In [ ]:
delivery.columns

In [ ]:
delivery.info()

### Data Analysis on Teams 

- Which Team has Won the most number of titles
- No of Each & Every Team
- Team Vs Team Comparisions
- Teams Along With Thier Favorite Venues
- Teams & Thier Homegrounds
- Teams along with thier win / lose margins
- Top player of each and every team
- Average Target Given by the team
- First Innings Prefference in case of toss winning
- Teams & Thier Tie Count.
- Top 5 most challenging teams for each team

In [ ]:
final_matches = match[match['match_type'] == 'Final']

In [ ]:
winners = final_matches['winner'].value_counts()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.bar(winners.index,height=winners.values,align='edge',color=['b','g','r','c','m','y'])
plt.title('Trophy Holders in IPL along with Frequency\n')
plt.show()

In [ ]:
team1_wins = match[match['winner'] == match['team1']]
team2_wins = match[match['winner'] == match['team2']]

In [ ]:
team_wins = (team2_wins['winner'].value_counts() + team1_wins['winner'].value_counts()).sort_values(ascending=False)

In [ ]:
plt.bar(team_wins.index,height=team_wins.values,align='edge',color=['b','g','r','c','m','y'])
plt.title('Team Along With Thier Winning Frequency\n')
plt.show()

In [ ]:
def compare_team(team1, team2):
    fig, ax = plt.subplots(2, 2, figsize=(26, 18))

    df = match[match['team1'].isin([team1, team2]) & match['team2'].isin([team1, team2])]
    cnt = df.shape[0]

    st = df['winner'].value_counts().sort_index()
    pl = df['player_of_match'].value_counts().sort_values(ascending=False).head(5)
    vn = df['venue'].value_counts().sort_values().tail(5)

    if st.index[0] == team1:
        leading_team = team1 if st[0] > (0.5) * cnt else team2
    else:
        leading_team = team2 if st[0] > (0.5) * cnt else team1

  
    ax[0, 0].pie(st.values, labels=st.index, autopct="%0.2f%%")
    ax[0, 0].set_title(f"Win % Comparison of {team1} and {team2}")

    ax[0, 1].bar(vn.index, vn.values)
    ax[0, 1].set_title(f"Top Venues for the Battle between {team1} and {team2}")
    ax[0, 1].tick_params(axis='x', labelrotation=30)

    ax[1, 1].bar(pl.index, pl.values, color=['r', 'b', 'g', 'y', 'c'])
    ax[1, 1].set_title("Top Players with Most Man of the Matches")
    ax[1, 1].tick_params(axis='x', labelrotation=45)
 
    toss_wins = df['toss_winner'].value_counts().reindex([team1, team2], fill_value=0)
    

    batting_preferences = df[df['toss_winner'].isin([team1, team2])].groupby('toss_winner')['toss_decision'].value_counts().unstack(fill_value=0).reindex([team1, team2], fill_value=0)

    toss_data = pd.DataFrame({
        'Toss Won': toss_wins,
        'Batting First Preference': batting_preferences.get('bat', 0)
    }).T


    ax[1, 0].bar(toss_data.columns, toss_data.loc['Toss Won'], label='Toss Won', color='skyblue')
    ax[1, 0].bar(toss_data.columns, toss_data.loc['Batting First Preference'], bottom=toss_data.loc['Toss Won'], label='Batting First Preference', color='salmon')
    ax[1, 0].set_title("Toss Wins and Batting Preferences")
    ax[1, 0].legend()
    ax[1, 0].tick_params(axis='x', labelrotation=45)

    plt.tight_layout()

    plt.show()


In [ ]:
compare_team('gt','srh')

In [ ]:
compare_team('mi','csk')

In [ ]:
def display_team_stats(match, display):
    final = []
    
    for team in (match['team1'].value_counts() + match['team2'].value_counts()).index:
        x = []
        x.append(team)

        filtered_df = match[(match['team1'] == team) | (match['team2'] == team)]

        matches_played = filtered_df.shape[0]
        match_won = (filtered_df['winner'] == team).sum()
        win_percentage = (match_won / matches_played * 100) if matches_played > 0 else 0
        trophies = ((filtered_df['match_type'] == 'Final') & (filtered_df['winner'] == team)).sum()
        
        x.extend([matches_played, match_won, round(win_percentage, 1), trophies])
        final.append(x)

    out = pd.DataFrame(final, columns=['Team Name', 'Matches Played', 'Matches Won', 'Win Percentage', 'Trophies'])
    out = out.sort_values(by='Trophies', ascending=False)
    
    if not display:
        return out
    

    fig, ax = plt.subplots(1, 2, figsize=(17, 5))

    ax[0].bar(out['Team Name'], out['Matches Played'], color='skyblue', label='Matches Played')
    ax[0].bar(out['Team Name'], out['Matches Won'], color='salmon', bottom=out['Matches Played'], label='Matches Won')
    ax[0].set_title("Matches Played Vs Won")
    ax[0].set_xlabel('Team Name')
    ax[0].set_ylabel('Count')
    ax[0].legend()
    

    ax[1].bar(out['Team Name'], out['Win Percentage'], color='skyblue')
    ax[1].set_title("Win Percentage")
    ax[1].set_xlabel('Team Name')
    ax[1].set_ylabel('Percentage')
    
    plt.xticks(rotation=90)  # Rotate team names for better readability
    plt.tight_layout()
    plt.show()


In [ ]:
display_team_stats(match,True)

In [ ]:
def top_players(team):
    
    def format_func(pct, all_vals):
        total = sum(all_vals)
        val = int(round(pct * total / 100.0))
        return f'{val}'


    filtered_df = match[((match['team1'] == team) | (match['team2'] == team)) & (match['winner'] == team)]
    top_players = filtered_df['player_of_match'].value_counts().sort_values(ascending=False).head()
    

    plt.figure(figsize=(7, 6))
    plt.pie(top_players.values, labels=top_players.index, autopct=lambda pct: format_func(pct, top_players.values))
    plt.title("Players With Most Player of the Match Awards for {}".format(team.title()))
    plt.show()


In [ ]:
top_players('gt')

In [ ]:
def top_defeats(team):
    filtered_df = match[(match['team1'].isin([team])) | (match['team2'].isin([team]))]
    filtered_df = filtered_df[filtered_df['winner'] != team]
    x = filtered_df['winner'].value_counts().sort_values(ascending=False).head()
    plt.pie(x.values,labels=x.index, autopct="%0.2f%%")
    plt.title("Top Teams which has defated {}".format(team))
    

In [ ]:
top_defeats('csk')

# Players Analysis

- Top Batsmen w.r.t the total runs scored  
- Top Bowlers w.r.t total wickets taken
- Boundries LeaderBoard
- SeasonWise Top Performers [Based on no. of man of the matches]

In [ ]:
delivery.rename({'match_id':'id'},axis=1,inplace=True)

In [ ]:
merged = match.merge(delivery,on='id')

In [ ]:
def fetchTopBatsman():
    batsmen = merged.groupby('batter')

In [ ]:
batsmen['batsman_runs'].aggregate({'sum'}).sort_values(by='sum',ascending=False).head(7)